<a href="https://colab.research.google.com/github/velasoka-repo/EVA4/blob/master/Session-8/EVA4_S8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Github Link](https://github.com/velasoka-repo/EVA4/tree/master/Session-8)

# Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
EVA4_PATH = "/content/gdrive/My Drive/Colab Notebooks/EVA4/"
import sys
if not EVA4_PATH in sys.path:
  sys.path.append(EVA4_PATH)
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Colab Notebooks/EVA4/']

# Python Module Import

In [0]:
import velasoka
device = velasoka.get_available_device()

# Generic Model Runner

In [0]:
def run_cnn(model, epochs, filename, scheduler_name=None):
  #dataset, data_loader
  net = model
  dataset = velasoka.get_cifar10_dataset()
  data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

  #optimizer, loss_function
  optimizer = velasoka.get_sgd_optimizer(model=net)
  scheduler = None
  if scheduler_name == "steplr":
    sceduler = velasoka.get_step_lr_scheduler(optimizer=optimizer, step_size=8)
  elif scheduler_name == "multisteplr":
    schduler = velasoka.get_multi_step_lr_scheduler(optimizer=optimizer, milestone=[10, 20, 30])
  
  loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

  #model_summary
  velasoka.model_summary(model=net, input_size=(3, 32, 32), device=device)
  #start cnn
  velasoka.start_cnn(model=net, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=epochs, scheduler=scheduler)

  #save model
  drive_file_path = velasoka.get_drive_path(folder="Session-8", filename=filename)
  velasoka.save_model(model=net, path=drive_file_path)

# ResNet18 Github Code

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

# ResNet18 without Linear(FC)

In [0]:
class ResNet_WOFC(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_WOFC, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=10, kernel_size=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = self.conv2(out)
        out = out.view(-1, 10)
        return out

# ResNet18 with Dropout & without Linear(FC)

In [0]:
class ResNet_WOFC_DO(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_WOFC_DO, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout2d(p=0.2)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.conv2 = nn.Conv2d(in_channels=512, out_channels=10, kernel_size=1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = self.dropout(out)
        out = F.relu(out)
        out = self.layer1(out)
        out = self.dropout(out)
        out = self.layer2(out)
        out = self.dropout(out)
        out = self.layer3(out)
        out = self.dropout(out)
        out = self.layer4(out)
        out = self.dropout(out)
        out = F.avg_pool2d(out, 4)
        out = self.conv2(out)
        out = out.view(-1, 10)
        return out

# ResNet18 without FC(Linear) :Result

In [0]:
def start_fn():
  #model
  net = ResNet_WOFC(BasicBlock, [2,2,2,2]).to(device)

  #dataset, data_loader
  dataset = velasoka.get_cifar10_dataset()
  data_loader = velasoka.get_cifar10_data_loader(dataset=dataset)

  #optimizer, loss_function
  optimizer = velasoka.get_sgd_optimizer(model=net)
  loss_fn = velasoka.get_ce_loss() #CrossEntropyLoss

  #model_summary
  velasoka.model_summary(model=net, input_size=(3, 32, 32), device=device)
  #start cnn
  velasoka.start_cnn(model=net, optimizer=optimizer, loss_fn=loss_fn, data_loader=data_loader, device=device, epochs=23)

  #save model
  drive_file_path = velasoka.get_drive_path(folder="Session-8", filename="resnet18-wo-fc.pt")
  velasoka.save_model(model=net, path=drive_file_path)

In [0]:
start_fn()

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

Training Batch=12499, loss=1.72461, Correct Prediction=24857/50000, Train Accuracy=49.71400: 100%|██████████| 12500/12500 [03:40<00:00, 56.73it/s]
Test Batch=2499, Correct Validation=6509/10000, Test Accuracy=65.09000: 100%|██████████| 2500/2500 [00:26<00:00, 93.42it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.00014, Correct Prediction=49942/50000, Train Accuracy=99.88400: 100%|██████████| 12500/12500 [03:35<00:00, 57.91it/s]
Test Batch=2499, Correct Validation=8466/10000, Test Accuracy=84.66000: 100%|██████████| 2500/2500 [00:25<00:00, 97.11it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 21


Training Batch=12499, loss=0.00074, Correct Prediction=49958/50000, Train Accuracy=99.91600: 100%|██████████| 12500/12500 [03:35<00:00, 57.88it/s]
Test Batch=2499, Correct Validation=8514/10000, Test Accuracy=85.14000: 100%|██████████| 2500/2500 [00:25<00:00, 96.78it/s] 
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 22


Training Batch=12499, loss=0.00024, Correct Prediction=49978/50000, Train Accuracy=99.95600: 100%|██████████| 12500/12500 [03:37<00:00, 57.48it/s]
Test Batch=2499, Correct Validation=8442/10000, Test Accuracy=84.42000: 100%|██████████| 2500/2500 [00:25<00:00, 96.65it/s] 


# ResNet18 without FC(Linear) & LR schedular :Result

In [0]:
model = ResNet_WOFC(BasicBlock, num_blocks=[2,2,2,2]).to(device)
run_cnn(model=model, epochs=25, filename="resnet18-wo-fc.pt", scheduler_name="multisteplr")


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

Training Batch=12499, loss=1.79418, Correct Prediction=25557/50000, Train Accuracy=51.11400: 100%|██████████| 12500/12500 [04:57<00:00, 42.06it/s]
Test Batch=2499, Correct Validation=6649/10000, Test Accuracy=66.49000: 100%|██████████| 2500/2500 [00:35<00:00, 71.14it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.00007, Correct Prediction=49961/50000, Train Accuracy=99.92200: 100%|██████████| 12500/12500 [05:06<00:00, 40.82it/s]
Test Batch=2499, Correct Validation=8527/10000, Test Accuracy=85.27000: 100%|██████████| 2500/2500 [00:34<00:00, 72.43it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 24


Training Batch=12499, loss=0.00030, Correct Prediction=49987/50000, Train Accuracy=99.97400: 100%|██████████| 12500/12500 [05:04<00:00, 41.01it/s]
Test Batch=2499, Correct Validation=8498/10000, Test Accuracy=84.98000: 100%|██████████| 2500/2500 [00:34<00:00, 71.75it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 25


Training Batch=12499, loss=0.00008, Correct Prediction=49991/50000, Train Accuracy=99.98200: 100%|██████████| 12500/12500 [05:04<00:00, 41.04it/s]
Test Batch=2499, Correct Validation=8499/10000, Test Accuracy=84.99000: 100%|██████████| 2500/2500 [00:34<00:00, 71.49it/s]


# ResNet18 without FC(Linear), with Dropout :Result

In [0]:
model = ResNet_WOFC_DO(BasicBlock, num_blocks=[2,2,2,2]).to(device)
run_cnn(model=model, epochs=25, filename="resnet18-wo-fc-do-steplr.pt", scheduler_name="steplr")

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
         Dropout2d-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
        BasicBlock-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,864
      BatchNorm2d-10           [-1, 64, 32, 32]             128
           Conv2d-11           [-1, 64, 32, 32]          36,864
      BatchNorm2d-12           [-1, 64, 32, 32]             128
       BasicBlock-13           [-1, 64, 32, 32]               0
        Dropout2d-14           [-1, 64,

Training Batch=12499, loss=1.69064, Correct Prediction=19102/50000, Train Accuracy=38.20400: 100%|██████████| 12500/12500 [04:53<00:00, 44.06it/s]
Test Batch=2499, Correct Validation=5227/10000, Test Accuracy=52.27000: 100%|██████████| 2500/2500 [00:33<00:00, 75.30it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.00310, Correct Prediction=49315/50000, Train Accuracy=98.63000: 100%|██████████| 12500/12500 [04:28<00:00, 46.58it/s]
Test Batch=2499, Correct Validation=8465/10000, Test Accuracy=84.65000: 100%|██████████| 2500/2500 [00:32<00:00, 77.83it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 24


Training Batch=12499, loss=0.00034, Correct Prediction=49305/50000, Train Accuracy=98.61000: 100%|██████████| 12500/12500 [04:37<00:00, 45.12it/s]
Test Batch=2499, Correct Validation=8459/10000, Test Accuracy=84.59000: 100%|██████████| 2500/2500 [00:33<00:00, 74.47it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 25


Training Batch=12499, loss=0.00300, Correct Prediction=49411/50000, Train Accuracy=98.82200: 100%|██████████| 12500/12500 [04:41<00:00, 44.40it/s]
Test Batch=2499, Correct Validation=8494/10000, Test Accuracy=84.94000: 100%|██████████| 2500/2500 [00:32<00:00, 76.23it/s]


# ResNet18 without FC(Linear) :Result

In [0]:
model = ResNet_WOFC_DO(BasicBlock, num_blocks=[2,2,2,2]).to(device)
run_cnn(model=model, epochs=25, filename="resnet18-wo-fc-steplr.pt", scheduler_name="steplr")

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
         Dropout2d-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
        BasicBlock-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,864
      BatchNorm2d-10           [-1, 64, 32, 32]             128
           Conv2d-11           [-1, 64, 32, 32]          36,864
      BatchNorm2d-12           [-1, 64, 32, 32]             128
       BasicBlock-13           [-1, 64, 32, 32]               0
        Dropout2d-14           [-1, 64,

Training Batch=12499, loss=1.89527, Correct Prediction=18944/50000, Train Accuracy=37.88800: 100%|██████████| 12500/12500 [05:10<00:00, 40.28it/s]
Test Batch=2499, Correct Validation=5192/10000, Test Accuracy=51.92000: 100%|██████████| 2500/2500 [00:34<00:00, 72.29it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.00094, Correct Prediction=49291/50000, Train Accuracy=98.58200: 100%|██████████| 12500/12500 [05:27<00:00, 38.18it/s]
Test Batch=2499, Correct Validation=8502/10000, Test Accuracy=85.02000: 100%|██████████| 2500/2500 [00:35<00:00, 69.57it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 24


Training Batch=12499, loss=0.01121, Correct Prediction=49299/50000, Train Accuracy=98.59800: 100%|██████████| 12500/12500 [05:25<00:00, 38.36it/s]
Test Batch=2499, Correct Validation=8461/10000, Test Accuracy=84.61000: 100%|██████████| 2500/2500 [00:35<00:00, 69.71it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 25


Training Batch=12499, loss=0.03212, Correct Prediction=49319/50000, Train Accuracy=98.63800: 100%|██████████| 12500/12500 [05:24<00:00, 38.55it/s]
Test Batch=2499, Correct Validation=8521/10000, Test Accuracy=85.21000: 100%|██████████| 2500/2500 [00:35<00:00, 70.86it/s]


# Default ResNet18 :Result

In [0]:
model = ResNet(BasicBlock, num_blocks=[2,2,2,2]).to(device)
run_cnn(model=model, epochs=10, filename="resnet18.pt")

Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/12500 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

Training Batch=12499, loss=1.85765, Correct Prediction=25284/50000, Train Accuracy=50.56800: 100%|██████████| 12500/12500 [04:29<00:00, 46.38it/s]
Test Batch=2499, Correct Validation=6662/10000, Test Accuracy=66.62000: 100%|██████████| 2500/2500 [00:32<00:00, 76.98it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 2


...

Training Batch=12499, loss=0.64203, Correct Prediction=47855/50000, Train Accuracy=95.71000: 100%|██████████| 12500/12500 [04:26<00:00, 46.84it/s]
Test Batch=2499, Correct Validation=8264/10000, Test Accuracy=82.64000: 100%|██████████| 2500/2500 [00:32<00:00, 77.70it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 9


Training Batch=12499, loss=0.00566, Correct Prediction=48384/50000, Train Accuracy=96.76800: 100%|██████████| 12500/12500 [04:28<00:00, 46.55it/s]
Test Batch=2499, Correct Validation=8271/10000, Test Accuracy=82.71000: 100%|██████████| 2500/2500 [00:32<00:00, 77.74it/s]
  0%|          | 0/12500 [00:00<?, ?it/s]


EPOCH: 10


Training Batch=12499, loss=0.31612, Correct Prediction=48979/50000, Train Accuracy=97.95800: 100%|██████████| 12500/12500 [04:27<00:00, 46.74it/s]
Test Batch=2499, Correct Validation=8192/10000, Test Accuracy=81.92000: 100%|██████████| 2500/2500 [00:32<00:00, 81.13it/s]
